In [3]:
import torch
import torchvision
from torch import optim
import torchvision.transforms as transforms
import numpy as np
from Cmodels import CMLP3, CMLP4
from tqdm.notebook import tqdm
import time
import argparse
import os

def eval():
    total = 0
    correct = 0
    model.clear_noise()
    model.clear_mask()
    with torch.no_grad():
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            # images = images.view(-1, 784)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correction = predictions == labels
            correct += correction.sum()
            total += len(correction)
    return correct/total

def eval_noise(var):
    total = 0
    correct = 0
    model.clear_noise()

    with torch.no_grad():
        model.set_noise(var)
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            # images = images.view(-1, 784)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correction = predictions == labels
            correct += correction.sum()
            total += len(correction)
    return (correct/total).cpu().numpy()

def train(epochs, header, verbose=False):
    best_acc = 0.0
    for i in range(epochs):
        running_loss = 0.
        running_l = 0.
        for images, labels in trainloader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            # images = images.view(-1, 784)
            outputs = model(images)
            loss = criteria(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        test_acc = eval()
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f"tmp_best_{header}.pt")
        if verbose:
            print(f"epoch: {i:-3d}, test acc: {test_acc:.4f}, loss: {running_loss / len(trainloader):.4f}")
        scheduler.step()

def GetSecond():
    model.clear_noise()
    optimizer.zero_grad()
    for images, labels in tqdm(trainloader):
        images, labels = images.to(device), labels.to(device)
        # images = images.view(-1, 784)
        outputs = model(images)
        model.Cbackward(outputs)

def str2bool(a):
    if a == "True":
        return True
    elif a == "False":
        return False
    else:
        raise NotImplementedError(f"{a}")


In [4]:
header = time.time()
header_timer = header
parent_path = "./"

device = torch.device("cpu")
# device = torch.device("cuda:0")

BS = 128

trainset = torchvision.datasets.MNIST(root='~/Private/data', train=True,
                                        download=False, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BS,
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='~/Private/data', train=False,
                                    download=False, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=BS,
                                            shuffle=False, num_workers=2)
model = CMLP3()
model.to(device)
model.push_S_device()
model.clear_noise()
model.clear_mask()
criteria = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [20])
pretrained = False
verbose = True
train_epoch = 10
noise_epoch = 1
noise_var = 0.2

if not pretrained:
    train(train_epoch, header, verbose)
    state_dict = torch.load(f"tmp_best_{header}.pt")
    model.load_state_dict(state_dict)
    torch.save(model.state_dict(), f"saved_B_{header}.pt")

    no_mask_acc_list = []
    state_dict = torch.load(f"saved_B_{header}.pt")
    print(f"No mask no noise: {eval():.4f}")
    model.load_state_dict(state_dict)
    model.clear_mask()
    loader = range(noise_epoch)
    for _ in loader:
        acc = eval_noise(noise_var)
        no_mask_acc_list.append(acc)
    print(f"No mask noise average acc: {np.mean(no_mask_acc_list):.4f}, std: {np.std(no_mask_acc_list):.4f}")
    torch.save(no_mask_acc_list, f"no_mask_list_{header}.pt")

epoch:   0, test acc: 0.9515, loss: 0.2952
epoch:   1, test acc: 0.9661, loss: 0.1522
epoch:   2, test acc: 0.9640, loss: 0.1265
epoch:   3, test acc: 0.9653, loss: 0.1134
epoch:   4, test acc: 0.9717, loss: 0.1054
epoch:   5, test acc: 0.9773, loss: 0.1005
epoch:   6, test acc: 0.9719, loss: 0.0899
epoch:   7, test acc: 0.9770, loss: 0.0882
epoch:   8, test acc: 0.9752, loss: 0.0831
epoch:   9, test acc: 0.9762, loss: 0.0825
No mask no noise: 0.9773
No mask noise average acc: 0.8101, std: 0.0000


In [5]:
GetSecond()

In [7]:
th = model.calc_S_grad_th(0.001)
model.set_mask(th, mode="th")

In [9]:
model.fc1.weightS.max()

tensor(21418.8262)

In [3]:

else:
    parent_path = args.model_path
    header = args.header
    no_mask_acc_list = torch.load(os.path.join(parent_path, f"no_mask_list_{header}.pt"))
    print(f"No mask noise average acc: {np.mean(no_mask_acc_list):.4f}, std: {np.std(no_mask_acc_list):.4f}")
    model.back_real(device)
    model.push_S_device()


state_dict = torch.load(os.path.join(parent_path, f"saved_B_{header}.pt"), map_location=device)
model.load_state_dict(state_dict)
model.back_real(device)
model.push_S_device()
criteria = SCrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [20])
model.clear_noise()
GetSecond()
print(f"S grad before masking: {model.fetch_S_grad().item():E}")

if args.use_mask:
    mask_acc_list = []
    th = model.calc_S_grad_th(args.mask_p)
    model.set_mask(th, mode="th")
    print(f"with mask no noise: {Seval():.4f}")
    # GetSecond()
    print(f"S grad after  masking: {model.fetch_S_grad().item():E}")
    GetSecond()
    print(f"S grad after  masking: {model.fetch_S_grad().item():E}")
    # loader = range(args.noise_epoch)
    # for _ in loader:
    #     acc = Seval_noise(args.noise_var)
    #     mask_acc_list.append(acc)
    # print(f"With mask noise average acc: {np.mean(mask_acc_list):.4f}, std: {np.std(mask_acc_list):.4f}")
    
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [20])
    STrain(args.fine_epoch, header_timer, args.verbose)

    if args.save_file:
        torch.save(model.state_dict(), f"saved_A_{header}_{header_timer}.pt")
    fine_mask_acc_list = []
    print(f"Finetune no noise: {Seval():.4f}")
    loader = range(args.noise_epoch)
    for _ in loader:
        acc = Seval_noise(args.noise_var)
        fine_mask_acc_list.append(acc)
    print(f"Finetune noise average acc: {np.mean(fine_mask_acc_list):.4f}, std: {np.std(fine_mask_acc_list):.4f}")
    model.clear_noise()
    GetSecond()
    print(f"S grad after finetune: {model.fetch_S_grad().item():E}")
    os.system(f"rm tmp_best_{header_timer}.pt")

AttributeError: 'CMLP4' object has no attribute 'graph'